# Estudo de Caso - Limpeza e Pré-Processamento de Dados com NumPy

### Contexto:

Imagine que em um determinado projeto de Ciência de Dados você receba um dataset extremamente complicado, contendo dados com muitas strings, caracteres especiais, problemas de encoding, datas  mal  formatadas,  números  e  textos  na  mesma  coluna,  url’s  contendo  Ids importantes para análise, valores ausentes, coluna que contém informação que deveria estar distribuída  em três ou  mais  colunas. E  como  se  não  bastasse  tudo  isso,  parte  dos  dados necessários para análise está em outro dataset, que deve ser combinado com o primeiro.

Seu trabalho seria limpar e pré-processar esse dataset, preparando-o para a sequência do processo de análise.

É exatamente este cenário que estamos reproduzindo no Estudo de Caso 1. A partir de dados complexos e com diversos problemas, iremos fazer um extenso trabalho de limpeza e pré-processamento. E tudo isso usando apenas o NumPy,  poderoso pacote da Linguagem Python para computação e processamento de dados.

#### Este Estudo de Caso traz uma quantidade incrível de conhecimento sobre manipulação de dados em Python.

## Definição do Problema e Fonte de Dados

#### Para este Estudo de Caso trabalharemos com dado reais disponíveis publicamente no link abaixo:
    
    https://www.openintro.org/data/index.php?data=loans_full_schema

Esse conjunto de dados representa milhares de empréstimos feitos por meio da plataforma Lending Club, que é uma
plataforma que permite que indivíduos emprestem para outros indivíduos.

Claro, nem todos os empréstimos são iguais. Alguém que fornece um baixo risco e que provavelmente vai pagar um
empréstimo terá mais facilidade em obter um empréstimo com uma taxa de juros baixa do que alguém que parece ser
mais arriscado.

E para as pessoas com alto risco de não pagar o empréstimo? Essas pessoas podem nem receber uma oferta de empréstimo, ou podem não aceitar uma oferta de empréstimo devido a uma alta taxa de juros. É importante ter em mente essa última parte, pois esse conjunto de dados representa  apenas  empréstimos  efetivamente  feitos,  ou  seja,  não  confunda  esses  dados  com pedidos de empréstimo!

Usamos como fonte de dados o dataset disponível no link acima, mas fizemos modificações nos dados para deixá-los ainda mais problemáticos. O dataset será fornecido a você junto com os demais arquivos do capítulo.

Além  disso  usaremos  um  dataset  com  cotação  do  dólar  em  relação  ao  Euro.  Extraímos uma pequena amostra de dados do site: https://finance.yahoo.com. O dataset será fornecido a você junto com os demais arquivos do capítulo.

## Objetivo:

Nosso trabalho é limpar e pré-processar os dados, deixando-os no formato ideal para um processo de análise posterior e várias decisões terão que ser tomadas no meio do caminho. Ao final do trabalho devemos salvar o dataset com os dados limpos e pré-processados

## Importando Pacotes e Configurando Impressão do NumPy

In [56]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
np.set_printoptions(suppress = True, linewidth = 200, precision = 2)

## Verificando a Codificação do Arquivo

In [57]:
import chardet

with open('datasets/dataset1.csv', 'rb') as f:
    result = chardet.detect(f.read())  # Lê uma amostra do arquivo
    print(result['encoding'])

windows-1251


## Carregando e Visualizando o Dataset

In [58]:
# Forma 1 (se não justificar o att dtype pois np.genfromtxt por padrão trata todo o arquivo como dados numéricos)

dados = np.genfromtxt("datasets/dataset1.csv",
                     delimiter = ';',
                     skip_header = 1,              # não carrega cabeçalho 
                     autostrip = True,
                     encoding = 'windows-1251'     # 'cp1252'
                     )
print(dados)


array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

In [77]:
# Forma 2 (atribuindo valor ao atributo dtype para tratar todo o arquivo como dados string)


dados2 = np.genfromtxt("datasets/dataset1.csv",
                     delimiter = ';',
                     dtype=str,              
                     autostrip = True,
                     encoding = 'windows-1251'     # 'cp1252'
                     )
print(dados2)

[['id' 'issue_d' 'loan_amnt' ... 'url' 'addr_state' 'total_pymnt']
 ['48010226' 'May-15' '35000.0' ... 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226' 'CA' '9452.96']
 ['57693261' '' '30000.0' ... 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261' 'NY' '4679.7']
 ...
 ['50415990' 'Jun-15' '10000.0' ... 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990' 'CA' '2185.64']
 ['46154151' 'Apr-15' '' ... 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151' 'OH' '3199.4']
 ['66055249' 'Dec-15' '10000.0' ... 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249' 'IL' '301.9']]
['id' 'issue_d' 'loan_amnt' 'loan_status' 'funded_amnt' 'term' 'int_rate' 'installment' 'grade' 'sub_grade' 'verification_status' 'url' 'addr_state' 'total_pymnt']


### Utilizando Pandas Apenas Para Visualizar o Dataset

In [82]:
import pandas as pd

# Convertendo o array NumPy para um DataFrame do Pandas
df = pd.DataFrame(dados)

# Exibindo o DataFrame
df.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,48010226.0,NaN,35000.0,NaN,35000.0,NaN,13.33,1184.86,NaN,NaN,NaN,NaN,NaN,9452.96
1,57693261.0,NaN,30000.0,NaN,30000.0,NaN,NaN,938.57,NaN,NaN,NaN,NaN,NaN,4679.70
2,59432726.0,NaN,15000.0,NaN,15000.0,NaN,NaN,494.86,NaN,NaN,NaN,NaN,NaN,1969.83
3,53222800.0,NaN,9600.0,NaN,9600.0,NaN,NaN,300.35,NaN,NaN,NaN,NaN,NaN,1793.68
4,57803010.0,NaN,8075.0,NaN,8075.0,NaN,19.19,296.78,NaN,NaN,NaN,NaN,NaN,1178.51
5,63398019.0,NaN,14400.0,NaN,14400.0,NaN,13.99,334.99,NaN,NaN,NaN,NaN,NaN,681.17
6,60850626.0,NaN,NaN,NaN,13000.0,NaN,24.99,381.50,NaN,NaN,NaN,NaN,NaN,1126.45
7,46816139.0,NaN,7500.0,NaN,7500.0,NaN,13.33,253.90,NaN,NaN,NaN,NaN,NaN,2025.65
8,63918356.0,NaN,8000.0,NaN,8000.0,NaN,NaN,250.29,NaN,NaN,NaN,NaN,NaN,497.07
9,63651616.0,NaN,7200.0,NaN,7200.0,NaN,NaN,241.18,NaN,NaN,NaN,NaN,NaN,512.57


Observe como várias colunas estão com o tipo NaN. Isso se deve a caracteres especiais no conjunto de dados e a forma como o NumPy carrega dados numéricos e do tipo string. Iremos resolver isso

In [83]:
# Convertendo o array NumPy para um DataFrame do Pandas 2 (visualizando dados carregados como string)
df2 = pd.DataFrame(dados2)

# Exibindo o DataFrame
df2.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,id,issue_d,loan_amnt,loan_status,funded_amnt,term,int_rate,installment,grade,sub_grade,verification_status,url,addr_state,total_pymnt
1,48010226,May-15,35000.0,Current,35000.0,36 months,13.33,1184.86,C,C3,Verified,https://www.lendingclub.com/browse/loanDetail....,CA,9452.96
2,57693261,,30000.0,Current,30000.0,36 months,юли.89,938.57,A,A5,Source Verified,https://www.lendingclub.com/browse/loanDetail....,NY,4679.7
3,59432726,Sep-15,15000.0,Current,15000.0,36 months,ное.53,494.86,B,B5,Verified,https://www.lendingclub.com/browse/loanDetail....,PA,1969.83
4,53222800,Jul-15,9600.0,Current,9600.0,36 months,юли.89,300.35,A,A5,Not Verified,https://www.lendingclub.com/browse/loanDetail....,OH,1793.68
5,57803010,Aug-15,8075.0,Current,8075.0,36 months,19.19,296.78,,E3,Source Verified,https://www.lendingclub.com/browse/loanDetail....,TX,1178.51
6,63398019,Oct-15,14400.0,Current,14400.0,60 months,13.99,334.99,,C4,Not Verified,https://www.lendingclub.com/browse/loanDetail....,AL,681.17
7,60850626,Sep-15,,Current,13000.0,60 months,24.99,381.5,F,F4,Source Verified,https://www.lendingclub.com/browse/loanDetail....,CA,1126.45
8,46816139,Apr-15,7500.0,Current,7500.0,36 months,13.33,253.9,C,C3,,https://www.lendingclub.com/browse/loanDetail....,GA,2025.65
9,63918356,Nov-15,8000.0,Current,8000.0,36 months,юли.89,250.29,A,A5,Source Verified,https://www.lendingclub.com/browse/loanDetail....,MA,497.07


### Tipo de Dados

In [60]:
print(type(dados))
print(type(df))

print(dados.shape)
print(df.shape)

<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>
(10000, 14)
(10000, 14)


## Verificando Valores Ausentes

In [61]:
# Visualizando novamente dataset e dados ausentes

dados.view()

array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

Observe como várias colunas estão com o tipo NaN. Isso se deve a caracteres especiais no conjunto de dados e a forma como o NumPy carrega dados numéricos e do tipo string. Iremos resolver isso

In [62]:
# Total de Valores Ausentes

np.isnan(dados).sum()

88005

## Tratando Valores Ausentes

In [63]:
# Vamos primeiramente retornar o maior valor + 1 ignorando valores NaN do dataset
# Usaremos este valor arbitrário para preencher os valores ausentes no momento da carga de dados de variáveis
# numéricas e depois tratamos esse valor como ausente

# Portanto pegamos o maior valor do conjunto de dados (sem considerar NaN), somamos 1 e gravamos em valor_coringa
# Vamos usar este valor para preeencher valores ausentes na hora de carregar variáveis numéricas

# Através do NumPy conseguimos passar por todo o conjunto de dados (memso que tenham outros tipos como chr) e ainda
# assim achar o maior valor numérico

valor_coringa = np.nanmax(dados) + 1
print(valor_coringa)

68616520.0


In [64]:
# Calculamos a média (variáveis numéricas) ignorando valores NaN por coluna
# Iremos usar isso para separar variáveis do tipo numéricas de variáveis do tipo string

media_ignorando_nan = np.nanmean(dados, axis = 0)
print(media_ignorando_nan)

[54015809.19         nan    15273.46         nan    15311.04         nan       16.62      440.92         nan         nan         nan         nan         nan     3143.85]


In [65]:
# Colunas do tipo Strings com valores ausentes

colunas_strings = np.argwhere(np.isnan(media_ignorando_nan)).squeeze()
colunas_strings

array([ 1,  3,  5,  8,  9, 10, 11, 12])

In [66]:
# Colunas do tipo Numérica com valores ausentes

colunas_numericas = np.argwhere(np.isnan(media_ignorando_nan) == False).squeeze()
colunas_numericas

array([ 0,  2,  4,  6,  7, 13])

#### Importando novamente o dataset, agora separando colunas do tipo String e colunas do tipo Numérica

In [67]:
# Carregando somente Colunas do Tipo String

arr_strings = np.genfromtxt("datasets/dataset1.csv",
                           delimiter = ';',
                           skip_header = 1,               # não carrega o cabeçalho
                           autostrip = True,
                           usecols = colunas_strings,
                           dtype = str,
                           encoding = 'windows-1251'
                           )

arr_strings

array([['May-15', 'Current', '36 months', ..., 'Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']], dtype='<U69')

In [68]:
# Carregando somente Colunas do Tipo Numérica

arr_numeric = np.genfromtxt("datasets/dataset1.csv",
                           delimiter = ';',
                           skip_header = 1,                      # não carrega o cabeçalho
                           autostrip = True,
                           usecols = colunas_numericas,
                           filling_values = valor_coringa,       # caso tenha valores NaN, ele substitui pelo 
                           encoding = 'windows-1251'             # valor_coringa
                           )

arr_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

#### Importando novamente o dataset, agora carregando apenas o nome das colunas

In [69]:
# Carrega nome das colunas

arr_nomes_colunas = np.genfromtxt("datasets/dataset1.csv",
                    delimiter = ';',
                    autostrip = True,
                    skip_footer = dados.shape[0],
                    dtype = str,
                    encoding = 'windows-1251' 
                    )

arr_nomes_colunas

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state', 'total_pymnt'], dtype='<U19')

In [70]:
# Separa cabeçalho de colunas numéricas e string

header_strings, header_numeric = arr_nomes_colunas[colunas_strings], arr_nomes_colunas[colunas_numericas]

print(header_strings)
print(header_numeric)

['issue_d' 'loan_status' 'term' 'grade' 'sub_grade' 'verification_status' 'url' 'addr_state']
['id' 'loan_amnt' 'funded_amnt' 'int_rate' 'installment' 'total_pymnt']


## Explicação Sobre o Processo de Tratamento de Valores ausentes

#### Verificando Valores Ausentes
Inicialmente, foi observado que o dataset contém uma quantidade significativa de valores NaN, indicando a presença de valores ausentes. Isso pode ser devido a várias razões, incluindo caracteres especiais, mistura de tipos de dados (numéricos e strings) em colunas, ou simplesmente dados faltantes.

#### Total de Valores Ausentes
A quantidade total de valores ausentes foi calculada usando np.isnan(dados).sum(), que forneceu uma visão geral da magnitude do problema.

### Estratégia para Tratar Valores Ausentes
A abordagem adotada para tratar valores ausentes consistiu em várias etapas:

#### Identificação de um Valor Coringa
Foi escolhido um valor coringa, definido como o maior valor no dataset mais um (np.nanmax(dados) + 1). Esse valor coringa foi utilizado para preencher temporariamente os valores ausentes nas colunas numéricas durante a carga do dataset, permitindo uma diferenciação clara entre os dados originalmente ausentes e outros valores numéricos.

#### Cálculo da Média Ignorando NaN
Para cada coluna, foi calculada a média, ignorando valores NaN (np.nanmean(dados, axis=0)). Isso ajudou a identificar quais colunas são puramente numéricas (pois a média de colunas contendo strings ou totalmente ausentes resultaria em NaN).

#### Separação de Colunas Numéricas e Strings
Com base na etapa anterior, as colunas foram classificadas como numéricas ou strings:

#### Colunas Strings:
Identificadas como aquelas cuja média resultou em NaN, indicando que não são puramente numéricas.

#### Colunas Numéricas
Colunas para as quais foi possível calcular uma média, indicando que são puramente numéricas.

#### Carregamento Separado de Dados Numéricos e Strings
Para tratar adequadamente os diferentes tipos de dados, o dataset foi carregado separadamente para colunas numéricas e strings:

#### Dados Numéricos
Carregados especificando usecols para selecionar apenas as colunas numéricas e filling_values para preencher valores ausentes com o valor coringa.

#### Dados de Strings
Carregados especificando usecols para selecionar apenas as colunas de strings, garantindo que o texto seja preservado corretamente.

#### Importação dos Nomes das Colunas
Os nomes das colunas foram importados separadamente, usando np.genfromtxt com parâmetros ajustados para carregar apenas o cabeçalho, fornecendo uma referência clara de quais dados cada coluna contém.

#### Separação dos Nomes das Colunas
Os nomes das colunas foram então divididos em dois grupos, correspondendo às colunas numéricas e de strings, com base na classificação feita anteriormente. Isso facilita a referência e a manipulação de colunas específicas nas etapas subsequentes.

#### Conclusão
Essas etapas formam a base para o tratamento de valores ausentes e a preparação dos dados para análise posterior. A diferenciação entre tipos de dados e o tratamento cuidadoso de valores ausentes são cruciais para garantir a integridade e a usabilidade dos dados em análises futuras.

# Função de Checkpoint

#### Checkpoint 1

Criação de uma função de Checkpoint para salvar os resultados feitos até aqui.

In [84]:
# Função
def checkpoint(file_name, checkpoint_header, checkpoint_data):
    np.savez(file_name, header = checkpoint_header, data = checkpoint_data)
    checkpoint_variable = np.load(file_name + ".npz")
    return(checkpoint_variable)

In [85]:
checkpoint_inicial = checkpoint("datasets/Checkpoint-Inicial", header_strings, arr_strings)

In [86]:
checkpoint_inicial['data']

array([['May-15', 'Current', '36 months', ..., 'Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']], dtype='<U69')

# Manipulando Colunas do Tipo String